In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pyngrok
!ngrok config add-authtoken 2SYBrn39spKZfP68C1ROobs0fBp_4bUQuiZ6wS3Qc1hom4x1E
!pip install python-multipart

import nest_asyncio
from fastapi import FastAPI
from pyngrok import ngrok
import uvicorn

from fastapi.responses import HTMLResponse
from fastapi import File, UploadFile

import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=486c76e6eb945f1312997e83bf66ab0913b7f306d3a75c727657037686ba100d
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import tensorflow as tf
model = tf.keras.models.load_model("/content/drive/MyDrive/model.h5")

In [40]:
file_path = '/content/drive/MyDrive/Colab Notebooks/skincancerModel/templates/new.html'

In [41]:
def getPrediction(img):

    classes = ['Actinic keratoses', 'Basal cell carcinoma',
               'Benign keratosis-like lesions', 'Dermatofibroma', 'Melanoma',
               'Melanocytic nevi', 'Vascular lesions']
    le = LabelEncoder()
    le.fit(classes)
    le.inverse_transform([2])


    # #Load model
    # my_model=load_model("models/model.h5")

    # SIZE = 32 #Resize to same size as training images
    # img_path = 'static/images/'+filename
    # img = np.asarray(Image.open(img_path).resize((SIZE,SIZE)))

    img = img/255.      #Scale pixel values

    img = np.expand_dims(img, axis=0)  #Get it tready as input to the network

    pred = model.predict(img) #Predict

    #Convert prediction to class name
    pred_class = le.inverse_transform([np.argmax(pred)])[0]
    print("Diagnosis is:", pred_class)
    return pred_class


In [42]:
# SIZE = 32

# @app.post("/upload")
# def upload(file: UploadFile = File(...)):
#     try:
#         contents = file.file.read()
#         # image = np.frombuffer(contents, np.uint8)
#         # image=image.resize(SIZE,SIZE)
#         # pred = getPrediction(image)
#         #return pred
#         with open(file.filename, 'wb') as f:
#             f.write(contents)
#     except Exception:
#         return {"message": "There was an error uploading the file"}
#     finally:
#         file.file.close()
#     return {"message": f"Successfully uploaded {file.filename}"}

In [43]:
app = FastAPI()

@app.get("/")
async def root():
    with open(file_path, "r") as file:
        content = file.read()
    return HTMLResponse(content=content)

In [44]:
# @app.get("/")
# async def get_homepage():
#     with open(file_path, "r") as file:
#         content = file.read()
#     return HTMLResponse(content=content)

In [45]:
@app.post("/upload")
def upload(file: UploadFile = File(...)):
    try:
        contents = file.file.read()
        image = np.asarray(Image.open(file.filename).resize((32,32)))
        pred = getPrediction(image)
        with open(file.filename, 'wb') as f:
            f.write(contents)
    except Exception as e:
        return {"hi iam abhishek gill"}
    finally:
        file.file.close()

    return {"message": f"Successfully uploaded {pred}"}
    # return {"Diagnosis for the following image is": pred}
    #return "123"
    #return redirect('/')


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app,port=8000)